In [2]:
import chess
import chess.svg
import chess.pgn
from tqdm import tqdm

In [3]:
piece_values = {
    chess.PAWN: 100,
    chess.ROOK: 500,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.QUEEN: 900,
    chess.KING: 20000
}
DEPTH_LIMIT = 3

In [4]:
#this functions displays the chess board. May add functions for options
def display_board(board):
    chess.svg.board(board)

In [5]:
def apply_move(board,move_str):
    """
    Applies a move in UCI/long algebraic notation (e.g. 'e2e4', 'e7e8q') 
    to the global chess.Board() instance.
    move_str: str, move in long algebraic/UCI notation
    Returns the updated board.
    """
    move = chess.Move.from_uci(move_str)  # parse the move
    if move in board.legal_moves:
        board.push(move)  # apply it
    else:
        raise ValueError(f"Illegal move: {move_str}")
    return board

In [6]:
chess_board = chess.Board()
NAME = "MUST WIN"
AUTHOR = "IAN WONG & ARNAV"

In [ ]:
def export_game(name, startingPos = None, moves = []):
    game = chess.pgn.Game()
    game.headers["Event"] = "Example Game"
    game.headers["Site"] = "MyComputer"
    game.headers["Date"] = "2025.09.02"
    game.headers["Round"] = "1"
    game.headers["White"] = "Player1"
    game.headers["Black"] = "Player2"
    if startingPos:
        game.setup(chess.Board(startingPos))
    node = None
    for move in moves:
        if not node: node = game.add_variation(chess.Move.from_uci(move))
        else: node = node.add_variation(chess.Move.from_uci(move))
    print(game, file=open(name + ".pgn", "w"), end="\n\n")

In [56]:
name = "first_line"
fen = "r1bqk2r/ppp1npbp/1n1pp1p1/8/2PPPP2/2NB1N2/PP4PP/R1BQ1RK1 b kq - 5 8"
moves = ["d1a4", "d8d7","f3e5","b6d5","e5f7","e7g8","f7d8","g8e7","d8f7","e7g8","f7d8","g8e7","d8f7","e7g8","f7d8","g8e7","d8f7","e7g8","f7d8","g8e7","d8b7","h8g8","b7d8","c8b7","d8f7","d7b5","f7h8","g8h8"]

export_game(name, startingPos=fen, moves = moves)

r1bqk2r/ppp1npbp/1n1pp1p1/8/2PPPP2/2NB1N2/PP4PP/R1BQ1RK1 b kq - 5 8
[Event "Example Game"]
[Site "MyComputer"]
[Date "2025.09.02"]
[Round "1"]
[White "Player1"]
[Black "Player2"]
[Result "*"]
[FEN "r1bqk2r/ppp1npbp/1n1pp1p1/8/2PPPP2/2NB1N2/PP4PP/R1BQ1RK1 b kq - 5 8"]
[SetUp "1"]

8... Qxa4 *
[Event "Example Game"]
[Site "MyComputer"]
[Date "2025.09.02"]
[Round "1"]
[White "Player1"]
[Black "Player2"]
[Result "*"]
[FEN "r1bqk2r/ppp1npbp/1n1pp1p1/8/2PPPP2/2NB1N2/PP4PP/R1BQ1RK1 b kq - 5 8"]
[SetUp "1"]

8... Qxa4 9. Qxd7+ *
[Event "Example Game"]
[Site "MyComputer"]
[Date "2025.09.02"]
[Round "1"]
[White "Player1"]
[Black "Player2"]
[Result "*"]
[FEN "r1bqk2r/ppp1npbp/1n1pp1p1/8/2PPPP2/2NB1N2/PP4PP/R1BQ1RK1 b kq - 5 8"]
[SetUp "1"]

8... Qxa4 9. Qxd7+ Nxe5 *
[Event "Example Game"]
[Site "MyComputer"]
[Date "2025.09.02"]
[Round "1"]
[White "Player1"]
[Black "Player2"]
[Result "*"]
[FEN "r1bqk2r/ppp1npbp/1n1pp1p1/8/2PPPP2/2NB1N2/PP4PP/R1BQ1RK1 b kq - 5 8"]
[SetUp "1"]

8... Qxa4 9. Qxd7+ Nx

In [9]:
def handleInput(command):
    parsedCommand = command.split()
    n = len(parsedCommand)
    i = 0
    if command == "quit": return -1
    elif command == "isready": return 0
    elif command == "uci": return 1
    elif command == "go": return 3
    elif parsedCommand[i] == "position":
        print("position")
        i+=1
        if parsedCommand[i] == "fen":
            fen = " ".join(parsedCommand[i+1:i+7])
            chess_board = chess.Board(fen)
            i += 7
        elif parsedCommand[i] == "startpos":
            chess_board = chess.Board()
            i += 1
        if parsedCommand[i] == "moves":
            i+=1
            while i < n:
                chess_board = apply_move(chess_board,parsedCommand[i])
                i+=1
                print(chess_board)
        return 2

In [ ]:
def minimax_root(board:chess.Board) -> chess.Move: 
    moves = list(board.legal_moves)
    bestMove, evaluation = moves[0], float("inf")
    for move in tqdm(moves):
        board.push(move)
        score = minimax(board, 0)
        if score < evaluation:
            evaluation = score
            bestMove = move
        board.pop()
    return bestMove
    
 
#uses the minimax algorithm at depth 5 to determine the best move on the chess board
#used the point system example on the blog: https://healeycodes.com/building-my-own-chess-engine
def minimax(board:chess.Board, depth = 0) -> float:
    moves = list(board.legal_moves)
    if depth == DEPTH_LIMIT or board.is_game_over(): 
        return calculate_score(board)
    #pick best score for the opponent
    if depth % 2:
        value = float("-inf")
        for move in moves:
            board.push(move)
            temp = minimax(board,depth+1)
            value = max(value,temp)
            board.pop()
        return value
    else:
        value = float("inf")
        for move in moves:
            board.push(move)
            temp = minimax(board,depth+1)
            value = min(value,temp)
            board.pop()
        return value
    

In [11]:
def handleOutput(command):
    if not command: print("readyok")
    elif command == 1:
        if NAME: print("id name " + NAME)
        if AUTHOR: print("id author " + AUTHOR)
        #add functionality for options that can be changed
        print("uciok")
    elif command == 3:
        move = determineBestMove(chess_board)
        print("bestmove " + move)
        apply_move(chess_board, move)
    
        

In [12]:
def calculate_score(board:chess):
    black_score,white_score = 0,0
    for square in chess.SQUARES:
        at_square = board.piece_at(square)
        if not at_square: continue
        if at_square.color == chess.WHITE:
            white_score += piece_values[at_square.piece_type]
        else:
            black_score += piece_values[at_square.piece_type]
    if board.turn == chess.WHITE: return white_score
    else: return black_score

In [13]:
def main():
    playing = True
    while playing:
        command = input()
        res = handleInput(command)
        handleOutput(res)
        display_board(chess_board)
        if res == -1:
            playing = False


In [143]:
main()

 quit


In [57]:
board = chess.Board()
name = "second_line"
moves = []
i=0
while not board.is_checkmate():
    bestMove = minimax_root(board)
    board.push(bestMove)
    chess.svg.board(board)
    moves.append(bestMove)
    i+=1
    if i == 10: break
export_game(name, moves)

100%|██████████| 21/21 [00:09<00:00,  2.27it/s]


AttributeError: 'list' object has no attribute 'split'